# <span style="color:green">Formation au Burkina Faso 2022</span> - Initiation à l’analyse de données Minion pour l'analyse de métagénome viraux

Created by J. Orjuela (DIADE-IRD), D. Filloux (PHIM-CIRAD) and A. Comte (PHIM-IRD) 

Septembre 2022

***

# <span style="color: #006E7F">Table of contents</span>
<a class="anchor" id="home"></a>
   

[TP1 - BASECALLING and QC](#data) 

[2. Basecalling](#guppy)

   * [2.1 Basecalling with `guppy`](#guppy)
    

[3. Quality Control on Long Reads](#qc)
   * [3.1 Quality Control of FASTQ with`Nanoplot`](#nanoplot)

</span>

***

### Our objectives in the following TP are : 
- explore the diversity of the metavirome of pineapple.
- reconstruct the complete genome sequencing of a novel member of the genus Vitivirus in the family Betaflexiviridae (subfamily Trivirinae) infecting pineapple.

# <span style="color:#006E7F">__TP1 - BASECALLING and QC__ <a class="anchor" id="data"></span>  
    
## <span style="color: #4CACBC;"> 1. Creating the folder, downloading data and so on</span>  

### <span style="color: #4CACBD;">  Data</span>
    

Before starting, please download special data created for this practical training. Data are available on the from I-Trop server.

This data is the total RNAs extracted from pineapple leaf samples collected in Reunion Island. Nanopore sequencing was performed using a MinION portable device and the cDNA-PCR Barcoding kit.

This original dataset contained more than 4M reads. We choose to sample it for this formation. 

`fast5` directory contains some electrical signals for basecalling step

`data.fastq` is a fastq file already basecalled 

In [8]:
mkdir -p ~/work/SG-ONT-2022/DATA
cd ~/work/SG-ONT-2022/DATA

# download fast5 sample data using for basecalling step
wget --no-check-certificat -rm -nH --cut-dirs=1 -r --no-parent --reject="index.html*" https://itrop.ird.fr/ont-training-2022/fast5/

--2022-09-05 15:39:22--  https://itrop.ird.fr/ont-training-2022/fast5/
Résolution de itrop.ird.fr (itrop.ird.fr)… 91.203.35.184
Connexion à itrop.ird.fr (itrop.ird.fr)|91.203.35.184|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 2586 (2,5K) [text/html]
Enregistre : «fast5/index.html.tmp»

fast5/index.html.tm 100%[===================>]   2,53K  --.-KB/s    ds 0s      

En-tête de dernière modification manquant — horodatage arrêté.
2022-09-05 15:39:23 (38,3 MB/s) - «fast5/index.html.tmp» enregistré [2586/2586]

Chargement de robots.txt ; veuillez ignorer les erreurs.
--2022-09-05 15:39:23--  https://itrop.ird.fr/robots.txt
Réutilisation de la connexion existante à itrop.ird.fr:443.
requête HTTP transmise, en attente de la réponse… 404 Not Found
2022-09-05 15:39:23 erreur 404 : Not Found.

Suppression de fast5/index.html.tmp puisqu’il devrait être rejeté.

--2022-09-05 15:39:23--  https://itrop.ird.fr/ont-training-2022/fast5/?C=N;O=D
Réutilisation de la 

In [9]:
ls ~/work/SG-ONT-2022/DATA

back.gif  blank.gif  fast5  unknown.gif


In [10]:
mkdir -p ~/work/SG-ONT-2022/DATA
cd ~/work/SG-ONT-2022/DATA

# download fastq sample data already basecalled
wget --no-check-certificat -rm -nH --cut-dirs=1 --reject="index.html*" https://itrop.ird.fr/ont-training-2022/data.fastq

--2022-09-05 15:44:03--  https://itrop.ird.fr/ont-training-2022/data.fastq
Résolution de itrop.ird.fr (itrop.ird.fr)… 91.203.35.184
Connexion à itrop.ird.fr (itrop.ird.fr)|91.203.35.184|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 1226812840 (1,1G)
Enregistre : «data.fastq»

data.fastq          100%[===================>]   1,14G  2,16MB/s    ds 11m 9s  

2022-09-05 15:55:12 (1,75 MB/s) - «data.fastq» enregistré [1226812840/1226812840]

Terminé — 2022-09-05 15:55:19 —
Temps total effectif : 11m 16s
Téléchargés : 1 fichiers, 1,1G en 11m 9s (1,75 MB/s)


## <span style="color: #4CACBC;"> 2. Basecalling <a class="anchor" id="guppy"></span> 

When DNA molecules are sequenced, electrical signals are stocked on fast5 format files.

These signals need to be converted on standard fastq files to post-analysis.

Several training dataset models are usually used to convert fast5 to fastq. 

### <span style="color: #4CACBC;"> 2.1 Basecalling with Guppy</span>


Guppy is a data processing toolkit that contains the 'Oxford Nanopore Technologies' basecalling algorithms, and several bioinformatic post-processing features.

Basecalling with guppy can be launch using guppy tool. 

Guppy takes fast5 raw read files and transform electrical signal in fastq files.

## Basecalling some available fast5 files 

In [1]:
#To see all the documentation of guppy:
  guppy_basecaller --help

: Guppy Basecalling Software, (C) Oxford Nanopore Technologies plc. Version 6.2.1+6588110a6, minimap2 version 2.22-r1101

Use of this software is permitted solely under the terms of the end user license agreement (EULA).By running, copying or accessing this software, you are demonstrating your acceptance of the EULA.
The EULA may be found in /opt/ont/guppy/bin

Usage:

With config file:
  guppy_basecaller -i <input path> -s <save path> -c <config file> [options]
With flowcell and kit name:
  guppy_basecaller -i <input path> -s <save path> --flowcell <flowcell name>
    --kit <kit name>
List supported flowcells and kits:
  guppy_basecaller --print_workflows

Command line parameters:
-c [ --config ]
	Configuration file for application.
-d [ --data_path ]
	Path to use for loading any data files the application requires.
-x [ --device ]
	Specify GPU device: 'auto', or 'cuda:<device_id>'.
-s [ --save_path ]
	Path to save output files.
--ping_url
	URL to send pings to.
--trace_domains_config

To run guppy you need to choose your configuration file accordingly to the flowcell and the kit you used for sequencing.

In [2]:
#List supported flowcells and kits:
  guppy_basecaller --print_workflows

Loading model version information, please wait ......................................................................
Available flowcell + kit combinations are:
flowcell       kit               barcoding config_name                    model version
FLO-PRO002     SQK-LSK112                  dna_r9.4.1_e8.1_hac_prom       2021-09-13_dna_r9.4.1_minion_promethion_384_ca963bcb
FLO-PRO002     SQK-LSK112-XL               dna_r9.4.1_e8.1_hac_prom       2021-09-13_dna_r9.4.1_minion_promethion_384_ca963bcb
FLO-PRO002     SQK-Q20EA                   dna_r9.4.1_e8.1_hac_prom       2021-09-13_dna_r9.4.1_minion_promethion_384_ca963bcb
FLO-PRO002     SQK-RAD112                  dna_r9.4.1_e8.1_hac_prom       2021-09-13_dna_r9.4.1_minion_promethion_384_ca963bcb
FLO-PRO002     SQK-NBD112-24     included  dna_r9.4.1_e8.1_hac_prom       2021-09-13_dna_r9.4.1_minion_promethion_384_ca963bcb
FLO-PRO002     SQK-NBD112-96     included  dna_r9.4.1_e8.1_hac_prom       2021-09-13_dna_r9.4.1_minion_promethion_38

3 types of config files:
- sup : high accuracy, very slow
- hac : medium accuracy, medium recources needes
- fast : poor accuracy, very fast

We recommend to basecaller yours dataset using a GPU graphic card to obtain results quickly.

Guppy can also demultiplex and trim adaptaters or barcodes.

#### Basecalling command line 

In [15]:
cd ~/work/SG-ONT-2022/DATA
rm output -rf

In [16]:
time guppy_basecaller -c dna_r9.4.1_450bps_fast.cfg -i fast5 -r -s output --min_qscore 6 --trim_adapters --detect_adapter --detect_mid_strand_adapter --pt_scaling --do_read_splitting
#--trim_adapters (trimming)
#--detect_adapter (detection of adaptaters)
#--pt_scaling  (polyT detection)
#--detect_mid_strand_adapter (in the middle of reads)
#--do_read_splitting (split if adaptater is detected in the middle of the read)

ONT Guppy basecalling software version 6.2.1+6588110a6, minimap2 version 2.22-r1101
config file:        /opt/ont/guppy/data/dna_r9.4.1_450bps_fast.cfg
model file:         /opt/ont/guppy/data/template_r9.4.1_450bps_fast.jsn
input path:         fast5
save path:          output
chunk size:         2000
chunks per runner:  160
minimum qscore:     6
records per file:   4000
num basecallers:    1
cpu mode:           ON
threads per caller: 4

Use of this software is permitted solely under the terms of the end user license agreement (EULA).By running, copying or accessing this software, you are demonstrating your acceptance of the EULA.
The EULA may be found in /opt/ont/guppy/bin
Found 1 fast5 files to process.
Init time: 80 ms

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************
Caller time: 190385 ms, Samples called: 33756550, samples/s: 177307
Finishing up any open output files.
Baseca

## <span style="color: #4CACBC;"> 3. Quality Control on Long Reads <a class="anchor" id="qc"></span> 


Calculating data quantity

In [ ]:
cd ~/work/SG-ONT-2022/DATA
pwd

Remember how fastq file is formatted. Each read is writen in fastq file using four lines :

* First line is the header. It has information about sequencer

* Second line is the sequence

* Third line start with + caracter and

* Four line contains quality score of each base

In [ ]:
head -n 4 data.fastq

Calculating how many reads are in the fastq file

In [ ]:
awk '{s++}END{print s/4}' data.fastq

Calculating how many bases were sequenced using seqtk

In [ ]:
seqtk seq -A data.fastq | grep -v ">" | wc -m

What is the sequencing depth?


### <span style="color: #4CACBD;"> 3.1 Quality Control of FASTQ with Nanoplot  <a class="anchor" id="nanoplot"></span> 

Control reads quality using Nanoplot. You can parameter this tool using --help.

In [ ]:
NanoPlot --help

Launch NanoPlot. You can launch NanoPlot using summaries or fastq files.

In [ ]:
# create a folder to save results
mkdir -p ~/work/SG-ONT-2022/QC
cd ~/work/SG-ONT-2022/QC

In [ ]:
###### run nanoplot 
NanoPlot -t 1 --fastq  ../DATA/data.fastq --outdir NANOPLOT

Check stats on created NanoStats file.

In [ ]:
cat NANOPLOT/NanoStats.txt

* What do you think about data? 

* Estimate coverage.

* What about reads quality qscore?

Observe NanoPlot-report.html.

To open it on jupyter you need to click on "trust HTML".

* what about this dataset?

###  Others complemental tools: 

https://github.com/wdecoster/NanoPlot#companion-scripts

* NanoComp: comparing multiple runs

* NanoStat: statistic summary report of reads or alignments

* NanoFilt: filtering and trimming of reads

* NanoLyse: removing contaminant reads (e.g. lambda control DNA) from fastq

* FiltLong : filtering long reads by quality https://github.com/rrwick/Filtlong


## <span style="color: #4CACBD;">4. BONUS: relaunch QC tools on the original dataset  <a class="anchor" id="bonus"></span> 

As descripted in 1.1 The dataset used previously is a sample of the original dataset.

In [ ]:
# download sequencing summary of the original data

mkdir -p ~/work/SG-ONT-2022/DATA/real_summaries/
cd ~/work/SG-ONT-2022/DATA/real_summaries/
wget --no-check-certificat -rm -nH --cut-dirs=1 --reject="index.html*" https://itrop.ird.fr/ont-training-2022/sequencing_summary.txt

Relaunch NanoComp and NanoPlot on this sequencing summary

In [ ]:
NanoPlot --summary sequencing_summary.txt --outdir ~/work/SG-ONT-2022/QC/NANOPLOT_real

Compare the outputs with the sampled ones.